# Learning MARL

### Verifying INstrallations

In [ ]:
import torch
import pybullet as p
from pettingzoo.butterfly import pistonball_v6
from stable_baselines3 import PPO
import numpy as np

print("PyTorch:", torch.__version__)
print('NumPy enabled:', p.isNumpyEnabled())
print("PettingZoo & Stable-Baselines3 & NumPy installed successfully!")

# Quick test: Initialize PyBullet
physics_client = p.connect(p.DIRECT)
p.disconnect()
print("PyBullet initialized successfully.")


In [ ]:
from pettingzoo.utils.conversions import aec_to_parallel
from supersuit import resize_v1
from stable_baselines3 import PPO
import supersuit as ss

# PettingZoo Environment
env = pistonball_v6.env()  # Create AEC environment
env = aec_to_parallel(env)  # Convert AEC to ParallelEnv
env = resize_v1(env, x_size=84, y_size=84)  # Resize observations
env = ss.pettingzoo_env_to_vec_env_v1(env)  # Convert to VecEnv
env = ss.concat_vec_envs_v1(env, num_vec_envs=1, base_class='stable_baselines3')  # Reduce num_vec_envs

# Initialize PPO with reduced n_steps and batch_size
model = PPO('MlpPolicy', env, verbose=1, n_steps=512, batch_size=64)

# Train for a few timesteps (testing purposes)
model.learn(total_timesteps=5000)

# Save Model
model.save("ppo_pistonball")

print("Training complete and model saved.")

In [ ]:
from pettingzoo.butterfly import pistonball_v6
from pettingzoo.utils.conversions import aec_to_parallel
from supersuit import resize_v1
import supersuit as ss
from stable_baselines3 import PPO
import numpy as np
import pygame
import time

# Initialize original PettingZoo environment (AEC)
env = pistonball_v6.env(render_mode='human')
env.reset()

env = aec_to_parallel(env)  # Convert AEC to ParallelEnv
env = resize_v1(env, x_size=84, y_size=84)  # Resize observations
# Convert to Parallel, then VecEnv for compatibility
parallel_env = ss.pettingzoo_env_to_vec_env_v1(env)
vec_env = ss.concat_vec_envs_v1(parallel_env, num_vec_envs=1, base_class='stable_baselines3')

# Load your trained model
model = PPO.load("ppo_pistonball")

# Reset environment
obs = vec_env.reset()

done = False
while not done:
    # Predict action from trained model
    action, _ = model.predict(obs, deterministic=True)
    
    # Take the predicted action
    obs, rewards, dones, info = vec_env.step(action)

    # Render the environment visually
    # env.reset()
    env.render()

    # Check if episode is complete
    done = np.all(dones)

    # Slow down the visualization for clear observation
    time.sleep(0.02)

env.close()


In [ ]:
# Understanding the environment, Piston Ball dataset and how has made observatiosna and rewards.

from pettingzoo.butterfly import pistonball_v6
from pettingzoo.utils.conversions import aec_to_parallel
from supersuit import resize_v1 


env = pistonball_v6.parallel_env(render_mode="human")
env = resize_v1(env, x_size=84, y_size=84)  # Resize observations
observations = env.reset()
# Load your trained model

while env.agents:
    # action, _ = model.predict(observations, deterministic=True)
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}

    observations, rewards, terminations, truncations, infos = env.step(actions)
    print("observation",rewards)
env.close()


In [ ]:
print("type", (observations.get('piston_1')).shape)
print("type", (observations.get('piston_1'))[83])


In [ ]:
from pettingzoo.butterfly import pistonball_v6

env = pistonball_v6.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

### This is the example given in the Petting ZOO site

In [ ]:
"""Basic code which shows what it's like to run PPO on the Pistonball env using the parallel API, this code is inspired by CleanRL.

This code is exceedingly basic, with no logging or weights saving.
The intention was for users to have a (relatively clean) ~200 line file to refer to when they want to design their own learning algorithm.

Author: Jet (https://github.com/jjshoots)
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from supersuit import color_reduction_v0, frame_stack_v1, resize_v1
from torch.distributions.categorical import Categorical

from pettingzoo.butterfly import pistonball_v6


class Agent(nn.Module):
    def __init__(self, num_actions):
        super().__init__()

        self.network = nn.Sequential(
            self._layer_init(nn.Conv2d(4, 32, 3, padding=1)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            self._layer_init(nn.Conv2d(32, 64, 3, padding=1)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            self._layer_init(nn.Conv2d(64, 128, 3, padding=1)),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Flatten(),
            self._layer_init(nn.Linear(128 * 8 * 8, 512)),
            nn.ReLU(),
        )
        self.actor = self._layer_init(nn.Linear(512, num_actions), std=0.01)
        self.critic = self._layer_init(nn.Linear(512, 1))

    def _layer_init(self, layer, std=np.sqrt(2), bias_const=0.0):
        torch.nn.init.orthogonal_(layer.weight, std)
        torch.nn.init.constant_(layer.bias, bias_const)
        return layer

    def get_value(self, x):
        return self.critic(self.network(x / 255.0))

    def get_action_and_value(self, x, action=None):
        hidden = self.network(x / 255.0)
        logits = self.actor(hidden)
        probs = Categorical(logits=logits)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action), probs.entropy(), self.critic(hidden)


def batchify_obs(obs, device):
    """Converts PZ style observations to batch of torch arrays."""
    # convert to list of np arrays
    obs = np.stack([obs[a] for a in obs], axis=0)
    # transpose to be (batch, channel, height, width)
    obs = obs.transpose(0, -1, 1, 2)
    # convert to torch
    obs = torch.tensor(obs).to(device)

    return obs


def batchify(x, device):
    """Converts PZ style returns to batch of torch arrays."""
    # convert to list of np arrays
    x = np.stack([x[a] for a in x], axis=0)
    # convert to torch
    x = torch.tensor(x).to(device)

    return x


def unbatchify(x, env):
    """Converts np array to PZ style arguments."""
    x = x.cpu().numpy()
    x = {a: x[i] for i, a in enumerate(env.possible_agents)}

    return x


if __name__ == "__main__":
    """ALGO PARAMS"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ent_coef = 0.1
    vf_coef = 0.1
    clip_coef = 0.1
    gamma = 0.99
    batch_size = 32
    stack_size = 4
    frame_size = (64, 64)
    max_cycles = 125
    total_episodes = 5

    """ ENV SETUP """
    env = pistonball_v6.parallel_env(
        render_mode="rgb_array", continuous=False, max_cycles=max_cycles
    )
    env = color_reduction_v0(env)
    env = resize_v1(env, frame_size[0], frame_size[1])
    env = frame_stack_v1(env, stack_size=stack_size)
    num_agents = len(env.possible_agents)
    num_actions = env.action_space(env.possible_agents[0]).n
    observation_size = env.observation_space(env.possible_agents[0]).shape

    """ LEARNER SETUP """
    agent = Agent(num_actions=num_actions).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=0.001, eps=1e-5)

    """ ALGO LOGIC: EPISODE STORAGE"""
    end_step = 0
    total_episodic_return = 0
    rb_obs = torch.zeros((max_cycles, num_agents, stack_size, *frame_size)).to(device)
    rb_actions = torch.zeros((max_cycles, num_agents)).to(device)
    rb_logprobs = torch.zeros((max_cycles, num_agents)).to(device)
    rb_rewards = torch.zeros((max_cycles, num_agents)).to(device)
    rb_terms = torch.zeros((max_cycles, num_agents)).to(device)
    rb_values = torch.zeros((max_cycles, num_agents)).to(device)

    """ TRAINING LOGIC """
    # train for n number of episodes
    for episode in range(total_episodes):
        # collect an episode
        with torch.no_grad():
            # collect observations and convert to batch of torch tensors
            next_obs, info = env.reset(seed=None)
            # reset the episodic return
            total_episodic_return = 0

            # each episode has num_steps
            for step in range(0, max_cycles):
                # rollover the observation
                obs = batchify_obs(next_obs, device)

                # get action from the agent
                actions, logprobs, _, values = agent.get_action_and_value(obs)

                # execute the environment and log data
                next_obs, rewards, terms, truncs, infos = env.step(
                    unbatchify(actions, env)
                )

                # add to episode storage
                rb_obs[step] = obs
                rb_rewards[step] = batchify(rewards, device)
                rb_terms[step] = batchify(terms, device)
                rb_actions[step] = actions
                rb_logprobs[step] = logprobs
                rb_values[step] = values.flatten()

                # compute episodic return
                total_episodic_return += rb_rewards[step].cpu().numpy()

                # if we reach termination or truncation, end
                if any([terms[a] for a in terms]) or any([truncs[a] for a in truncs]):
                    end_step = step
                    break

        # bootstrap value if not done
        with torch.no_grad():
            rb_advantages = torch.zeros_like(rb_rewards).to(device)
            for t in reversed(range(end_step)):
                delta = (
                    rb_rewards[t]
                    + gamma * rb_values[t + 1] * rb_terms[t + 1]
                    - rb_values[t]
                )
                rb_advantages[t] = delta + gamma * gamma * rb_advantages[t + 1]
            rb_returns = rb_advantages + rb_values

        # convert our episodes to batch of individual transitions
        b_obs = torch.flatten(rb_obs[:end_step], start_dim=0, end_dim=1)
        b_logprobs = torch.flatten(rb_logprobs[:end_step], start_dim=0, end_dim=1)
        b_actions = torch.flatten(rb_actions[:end_step], start_dim=0, end_dim=1)
        b_returns = torch.flatten(rb_returns[:end_step], start_dim=0, end_dim=1)
        b_values = torch.flatten(rb_values[:end_step], start_dim=0, end_dim=1)
        b_advantages = torch.flatten(rb_advantages[:end_step], start_dim=0, end_dim=1)

        # Optimizing the policy and value network
        b_index = np.arange(len(b_obs))
        clip_fracs = []
        for repeat in range(3):
            # shuffle the indices we use to access the data
            np.random.shuffle(b_index)
            for start in range(0, len(b_obs), batch_size):
                # select the indices we want to train on
                end = start + batch_size
                batch_index = b_index[start:end]

                _, newlogprob, entropy, value = agent.get_action_and_value(
                    b_obs[batch_index], b_actions.long()[batch_index]
                )
                logratio = newlogprob - b_logprobs[batch_index]
                ratio = logratio.exp()

                with torch.no_grad():
                    # calculate approx_kl http://joschu.net/blog/kl-approx.html
                    old_approx_kl = (-logratio).mean()
                    approx_kl = ((ratio - 1) - logratio).mean()
                    clip_fracs += [
                        ((ratio - 1.0).abs() > clip_coef).float().mean().item()
                    ]

                # normalize advantaegs
                advantages = b_advantages[batch_index]
                advantages = (advantages - advantages.mean()) / (
                    advantages.std() + 1e-8
                )

                # Policy loss
                pg_loss1 = -b_advantages[batch_index] * ratio
                pg_loss2 = -b_advantages[batch_index] * torch.clamp(
                    ratio, 1 - clip_coef, 1 + clip_coef
                )
                pg_loss = torch.max(pg_loss1, pg_loss2).mean()

                # Value loss
                value = value.flatten()
                v_loss_unclipped = (value - b_returns[batch_index]) ** 2
                v_clipped = b_values[batch_index] + torch.clamp(
                    value - b_values[batch_index],
                    -clip_coef,
                    clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[batch_index]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()

                entropy_loss = entropy.mean()
                loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
        var_y = np.var(y_true)
        explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

        print(f"Training episode {episode}")
        print(f"Episodic Return: {np.mean(total_episodic_return)}")
        print(f"Episode Length: {end_step}")
        print("")
        print(f"Value Loss: {v_loss.item()}")
        print(f"Policy Loss: {pg_loss.item()}")
        print(f"Old Approx KL: {old_approx_kl.item()}")
        print(f"Approx KL: {approx_kl.item()}")
        print(f"Clip Fraction: {np.mean(clip_fracs)}")
        print(f"Explained Variance: {explained_var.item()}")
        print("\n-------------------------------------------\n")

    """ RENDER THE POLICY """
    env = pistonball_v6.parallel_env(render_mode="human", continuous=False)
    env = color_reduction_v0(env)
    env = resize_v1(env, 64, 64)
    env = frame_stack_v1(env, stack_size=4)

    agent.eval()

    with torch.no_grad():
        # render 5 episodes out
        for episode in range(5):
            obs, infos = env.reset(seed=None)
            obs = batchify_obs(obs, device)
            terms = [False]
            truncs = [False]
            while not any(terms) and not any(truncs):
                actions, logprobs, _, values = agent.get_action_and_value(obs)
                obs, rewards, terms, truncs, infos = env.step(unbatchify(actions, env))
                obs = batchify_obs(obs, device)
                terms = [terms[a] for a in terms]
                truncs = [truncs[a] for a in truncs]

In [ ]:
import pybullet as p
import pybullet_data
import time

# Initialize PyBullet
physics_client = p.connect(p.GUI)  # or p.DIRECT for non-GUI
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Setup world and physics
p.setGravity(0, 0, -9.8)
plane_id = p.loadURDF("plane.urdf")

# Load robot agents
robot_start_positions = [[0, 0, 0.2], [1, 0, 0.2]]
robot_ids = []
for pos in robot_start_positions:
    robot_id = p.loadURDF("r2d2.urdf", pos)
    robot_ids.append(robot_id)

# Simulation loop (simple demo)
for step in range(500):
    p.stepSimulation()
    time.sleep(1./240.)

p.disconnect()


## Structuring Custom Envionment 
#### UsinG ChatGPT and It has many loop holes

In [25]:
from pettingzoo import AECEnv
from pettingzoo.utils import wrappers
from pettingzoo.utils.agent_selector import agent_selector

from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import time


class GrabAndMoveEnv(AECEnv):
    metadata = {"render_modes": ["human"], "name": "grab_and_move_v0",  "is_parallelizable": True  }

    def __init__(self, render_mode=None):
        super().__init__()
        self.possible_agents = ["agent_0", "agent_1"]
        self.agents = self.possible_agents[:]
        self.render_mode = render_mode 
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}
        self.agent_idx = 0
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}

        self.action_spaces = {agent: spaces.Discrete(5) for agent in self.agents}
        self.observation_spaces = {
            agent: spaces.Box(low=-3.0, high=3.0, shape=(6,), dtype=np.float32)
            for agent in self.agents
        }

    def reset(self, seed=None, options=None):
        self.agents = self.possible_agents[:]
        self.agent_idx = 0

        if p.getConnectionInfo()['isConnected']:
            p.disconnect()
        if p.isConnected():
            p.disconnect()

        if self.render_mode == "human":
            self.physicsClient = p.connect(p.GUI)
        else:
            self.physicsClient = p.connect(p.DIRECT)

        # self.physicsClient = p.connect(p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.resetSimulation()
        p.setGravity(0, 0, -9.8)

        p.loadURDF("plane.urdf")
        self.create_walls()

        self.cube = self.create_cube_block(-2, 0)
        self.agent_bodies = [
            self.create_agent(-2.5, 0.5, [1, 0, 0, 1]),
            self.create_agent(-2.5, -0.5, [0, 0, 1, 1])
        ]
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

        self.constraints = [None, None]
        self.rewards = {agent: 0 for agent in self.agents}
        # self.dones = {agent: False for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}

        self.infos = {agent: {} for agent in self.agents}
        self._cumulative_rewards = {agent: 0.0 for agent in self.agents}

    def create_walls(self):
        def wall(x, y, hx, hy):
            shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[hx, hy, 0.2])
            visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[hx, hy, 0.2], rgbaColor=[0.8, 0.8, 0.8, 1])
            return p.createMultiBody(0, shape, visual, [x, y, 0.2])
        wall(0, 3.0, 3.0, 0.05)
        wall(0, -3.0, 3.0, 0.05)
        wall(-3.0, 0, 0.05, 3.0)
        wall(3.0, 0, 0.05, 3.0)
        wall(0, 1.7, 0.05, 1.3)
        wall(0, -1.7, 0.05, 1.3)

    def create_cube_block(self, x, y):
        shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1])
        visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.1, 0.1, 0.1], rgbaColor=[0, 1, 0, 1])
        return p.createMultiBody(1.0, shape, visual, [x, y, 0.1])

    def create_agent(self, x, y, color):
        shape = p.createCollisionShape(p.GEOM_SPHERE, radius=0.05)
        visual = p.createVisualShape(p.GEOM_SPHERE, radius=0.05, rgbaColor=color)
        return p.createMultiBody(0.1, shape, visual, [x, y, 0.05])

    def step(self, action):
        agent = self.agents[self.agent_idx]
        idx = self.agent_name_mapping[agent]
        vx, vy = 0, 0
        if action == 1: vy = 0.5
        elif action == 2: vy = -0.5
        elif action == 3: vx = -0.5
        elif action == 4: vx = 0.5
        p.resetBaseVelocity(self.agent_bodies[idx], linearVelocity=[vx, vy, 0])

        for _ in range(5):
            p.stepSimulation()

        self.rewards[agent] = -0.01

        cube_pos, _ = p.getBasePositionAndOrientation(self.cube)
        if cube_pos[0] > 2.0:
            # self.dones = {a: True for a in self.agents}
            self.terminations = {a: True for a in self.agents}

            self.rewards[agent] = 1.0

        self.agent_idx = (self.agent_idx + 1) % len(self.agents)
        self.agent_selection = self._agent_selector.next()


    def observe(self, agent):
        idx = self.agent_name_mapping[agent]
        agent_pos, _ = p.getBasePositionAndOrientation(self.agent_bodies[idx])
        cube_pos, _ = p.getBasePositionAndOrientation(self.cube)
        other_idx = 1 - idx
        other_pos, _ = p.getBasePositionAndOrientation(self.agent_bodies[other_idx])
        return np.array(agent_pos[:2] + cube_pos[:2] + other_pos[:2], dtype=np.float32)

    def render(self):
        if self.render_mode == "human":
            pass
        pass  # Not needed with PyBullet GUI

    def close(self):
        p.disconnect()

    def agent_iter(self, max_iter=1000):
        from pettingzoo.utils.agent_selector import agent_selector
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()
        for _ in range(max_iter):
            yield self.agent_selection
            self.agent_selection = self._agent_selector.next()

    def state(self):
        return np.concatenate([self.observe(agent) for agent in self.agents])


In [ ]:
self.possible_agents = ["agent_0", "agent_1"]
        self.agents = self.possible_agents[:]
        self.render_mode = render_mode 
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}
        self.agent_idx = 0
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}

        self.action_spaces = {agent: spaces.Discrete(5) for agent in self.agents}
        self.observation_spaces = {
            agent: spaces.Box(low=-3.0, high=3.0, shape=(6,), dtype=np.float32)
            for agent in self.agents
        }

In [32]:
aa= GrabAndMoveEnv()
aa.reset()
nex=aa._agent_selector.next()
print("Agent : ", nex)
print("Actopn : ", aa.terminations)

Agent :  agent_1
Actopn :  {'agent_0': False, 'agent_1': False}


In [36]:
agent = aa.agents[aa.agent_idx]
aa.terminations[agent]=True
print("Actopn : ", aa.terminations)

Actopn :  {'agent_0': True, 'agent_1': False, 0: True}


In [58]:
from utils.physics_helpers import create_wall

In [59]:
create_wall(x_center=0,    y_center= 3.0, half_ext_x=3.0, half_ext_y=0.05)  # Top


11

In [46]:
agn=0
Fun_{f'agn'} = 9090
print("Agent : ", agn)

SyntaxError: invalid syntax (1293115488.py, line 2)

### Training

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from pettingzoo.utils.conversions import aec_to_parallel
from supersuit import pettingzoo_env_to_vec_env_v1, concat_vec_envs_v1
# from grab_and_move_env import GrabAndMoveEnv

# 1️⃣ Create and convert PettingZoo env to parallel
parallel_env = aec_to_parallel(GrabAndMoveEnv())

# 2️⃣ Convert to vectorized env
vec_env = pettingzoo_env_to_vec_env_v1(parallel_env)
vec_env = concat_vec_envs_v1(vec_env, num_vec_envs=4, num_cpus=1, base_class='stable_baselines3')

# 3️⃣ Add VecMonitor for logging
vec_env = VecMonitor(vec_env)

# 4️⃣ Instantiate PPO model
model = PPO("MlpPolicy", vec_env, verbose=1, tensorboard_log="./ppo_grab_and_move_tensorboard/")

# 5️⃣ Train the agent
model.learn(total_timesteps=100000)

# 6️⃣ Save the trained model
model.save("ppo_grab_and_move")
print("Model trained and saved as 'ppo_grab_and_move'")


Using cpu device
Logging to ./ppo_grab_and_move_tensorboard/PPO_2


d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\pettingzoo\utils\env.py:120: UserWarning: Your environment should override the observation_space function. Attempting to use the observation_spaces dict attribute.
  warnings.warn(
d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\pettingzoo\utils\env.py:132: UserWarning: Your environment should override the action_space function. Attempting to use the action_spaces dict attribute.
  warnings.warn(
d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:78: UserWarning: The `render_mode` attribute is not defined in your environment. It will be set to None.
  warnings.warn("The `render_mode` attribute is not defined in your environment. It will be set to None.")


------------------------------
| time/              |       |
|    fps             | 6402  |
|    iterations      | 1     |
|    time_elapsed    | 2     |
|    total_timesteps | 16384 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 2904        |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.020225247 |
|    clip_fraction        | 0.24        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.59       |
|    explained_variance   | 0.922       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0152      |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00791    |
|    value_loss           | 0.000891    |
-----------------------------------------
---------------------------

### Evaluation

In [13]:
from stable_baselines3 import PPO
from pettingzoo.utils.conversions import aec_to_parallel
from supersuit import pettingzoo_env_to_vec_env_v1, concat_vec_envs_v1
# from grab_and_move_env import GrabAndMoveEnv
import time

# 1️⃣ Load the trained PPO model
model = PPO.load("ppo_grab_and_move")

# 2️⃣ Create the PettingZoo parallel environment with GUI rendering
raw_env = GrabAndMoveEnv(render_mode="human")
parallel_env = aec_to_parallel(raw_env)

# 3️⃣ Convert to SB3-compatible VecEnv (1 instance only for GUI)
vec_env = pettingzoo_env_to_vec_env_v1(parallel_env)
vec_env = concat_vec_envs_v1(vec_env, num_vec_envs=1, num_cpus=1, base_class='stable_baselines3')

# 4️⃣ Reset the environment
obs = vec_env.reset()

# 5️⃣ Evaluation loop
for step in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)

    print(f"Step {step}, Reward: {reward}")
    time.sleep(0.05)  # slows down for GUI view

print("✅ Evaluation finished.")


Exception ignored in: <function VectorEnv.__del__ at 0x00000225D82D2020>
Traceback (most recent call last):
  File "d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\gymnasium\vector\vector_env.py", line 342, in __del__
    self.close()
  File "d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\supersuit\vector\markov_vector_wrapper.py", line 112, in close
    return self.par_env.close()
           ^^^^^^^^^^^^^^^^^^^^
  File "d:\UMDCP\SEM4\ENPM690\Final Project\Multi_Agent_Reinforcement_Learning\marl-env\Lib\site-packages\pettingzoo\utils\conversions.py", line 233, in close
    return self.aec_env.close()
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\keyur\AppData\Local\Temp\ipykernel_25016\3585985279.py", line 132, in close
pybullet.error: Not connected to physics server.
Exception ignored in: <function VectorEnv.__del__ at 0x00000225D82D2020>
Traceback (most recent call last):
  File "d

Step 0, Reward: [-0.02 -0.01]
Step 1, Reward: [-0.02 -0.02]
Step 2, Reward: [-0.02 -0.02]
Step 3, Reward: [-0.02 -0.02]
Step 4, Reward: [-0.02 -0.02]
Step 5, Reward: [-0.02 -0.02]
Step 6, Reward: [-0.02 -0.02]
Step 7, Reward: [-0.02 -0.02]
Step 8, Reward: [-0.02 -0.02]
Step 9, Reward: [-0.02 -0.02]
Step 10, Reward: [-0.02 -0.02]
Step 11, Reward: [-0.02 -0.02]
Step 12, Reward: [-0.02 -0.02]
Step 13, Reward: [-0.02 -0.02]
Step 14, Reward: [-0.02 -0.02]
Step 15, Reward: [-0.02 -0.02]
Step 16, Reward: [-0.02 -0.02]
Step 17, Reward: [-0.02 -0.02]
Step 18, Reward: [-0.02 -0.02]
Step 19, Reward: [-0.02 -0.02]
Step 20, Reward: [-0.02 -0.02]
Step 21, Reward: [-0.02 -0.02]
Step 22, Reward: [-0.02 -0.02]
Step 23, Reward: [-0.02 -0.02]
Step 24, Reward: [-0.02 -0.02]
Step 25, Reward: [-0.02 -0.02]
Step 26, Reward: [-0.02 -0.02]
Step 27, Reward: [-0.02 -0.02]
Step 28, Reward: [-0.02 -0.02]
Step 29, Reward: [-0.02 -0.02]
Step 30, Reward: [-0.02 -0.02]
Step 31, Reward: [-0.02 -0.02]
Step 32, Reward: [

In [ ]:
from pettingzoo import AECEnv
from pettingzoo.utils import wrappers
from pettingzoo.utils.agent_selector import agent_selector

from gymnasium import spaces
import numpy as np
import pybullet as p
import pybullet_data
import time

class MyCustomEnvironment(AECEnv):
   
    metadata = {"render_modes": ["human"], "name": "grab_and_move_v0",  "is_parallelizable": True  }

    def __init__(self, render_mode=None):
        super().__init__()
        self.possible_agents = ["agent_0", "agent_1"]
        self.agents = self.possible_agents[:]
        self.render_mode = render_mode 
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}
        self.agent_idx = 0
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}

        self.action_spaces = {agent: spaces.Discrete(8) for agent in self.agents}
        self.observation_spaces = {
            agent: spaces.Box(low=-3.0, high=3.0, shape=(6,), dtype=np.float32)
            for agent in self.agents
        }


    def reset(self, seed=None, options=None):
        # Killing the previous connection if it exists
        if p.isConnected():
            p.disconnect()
        # Connect to PyBullet
        if self.render_mode == "human":
            self.physicsClient = p.connect(p.GUI)
        else:
            self.physicsClient = p.connect(p.DIRECT)
        # Getting Number of Agents
        self.agents = self.possible_agents[:]
        self.agent_idx = 0
        # Set Uo additional search path for URDF files
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        # Create a plane (floor)
        plane_id = p.loadURDF("plane.urdf")
        p.setGravity(0, 0, -9.8) # gravity setting

        # Create outer boundary walls (6x6 area, from -3 to +3 in x,y)
        self.create_wall(x_center=0,    y_center= 3.0, half_ext_x=3.0, half_ext_y=0.05)  # Top
        self.create_wall(x_center=0,    y_center=-3.0, half_ext_x=3.0, half_ext_y=0.05)  # Bottom
        self.create_wall(x_center=-3.0, y_center= 0.0, half_ext_x=0.05, half_ext_y=3.0)  # Left
        self.create_wall(x_center= 3.0, y_center= 0.0, half_ext_x=0.05, half_ext_y=3.0)  # Right

        # Create dividing wall with a slit in the middle
        self.create_wall(x_center=0, y_center= 1.7, half_ext_x=0.05, half_ext_y=1.3)  # Upper
        self.create_wall(x_center=0, y_center=-1.7, half_ext_x=0.05, half_ext_y=1.3)  # Lower

        # Create the cube block
        self.cube_id = self.create_cube_block(pos_x=-2, pos_y=0, size=0.2, mass=1.0)

        # Create two agents (small spheres)
        self.agent1_id = self.create_point_agent(pos_x=-2.5, pos_y= 0.5, radius=0.05, mass=0.1, color=[1,0,0,1])  # red
        self.agent2_id = self.create_point_agent(pos_x=-2.5, pos_y=-0.5, radius=0.05, mass=0.1, color=[0,0,1,1])  # blue
        # Tracking whether each agent is currently grabbing the cube
        self.agent1_grabbing = False
        self.agent2_grabbing = False

        # Store constraint IDs for each agent if they grab the cube
        self.agent1_constraint_id = None
        self.agent2_constraint_id = None

        # Parameters
        self.grab_distance_threshold = 1.3  # max distance for an agent to be able to grab the cube
        self.agent_speed = 0.50             # speed for each agent (adjust as desired)
        self.rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}

        self.infos = {agent: {} for agent in self.agents}
        self._cumulative_rewards = {agent: 0.0 for agent in self.agents}
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()


    def step(self, actions):
        agent = self.agents[self.agent_idx]
        idx = self.agent_name_mapping[agent]
        movement_map = {
            0: (0.1, 0.0),   # forward
            1: (-0.1, 0.0),  # backward
            2: (0.0, -0.1),  # left
            3: (0.0, 0.1)    # right
        }

        attach = actions >= 4
        move_action = actions % 4
        dx, dy = movement_map[move_action]

        # Move agent
        p.resetBaseVelocity(self.agent, [dx, dy, 0])
        for _ in range(5):
            p.stepSimulation()

        # Check distance
        agent_pos = np.array(p.getBasePositionAndOrientation(self.agent)[0])
        cube_pos = np.array(p.getBasePositionAndOrientation(self.cube)[0])
        dist = np.linalg.norm(agent_pos[:2] - cube_pos[:2])

        # Handle attachment logic
        if attach and idx==0 and dist < self.grab_distance_threshold:
            self.attached = True
            self.agent1_grabbing = True
            self.agent1_constraint_id = self.create_fixed_constraint(
                            self.agent1_id, self.cube_id
                        )
        elif (not attach or dist > self.grab_distance_threshold) and idx==0:
            if self.attached==True:
            
                self.attached = False
                self.agent1_grabbing = False
                p.removeConstraint(self.agent1_constraint_id)
                self.agent1_constraint_id = None

        elif attach and idx==1 and dist < self.grab_distance_threshold:
            self.attached = True
            self.agent2_grabbing = True
            self.agent2_constraint_id = self.create_fixed_constraint(
                            self.agent2_id, self.cube_id
                        )
        elif (not attach or dist > self.grab_distance_threshold) and idx==1:
            if self.attached==True:
                self.attached = False
                self.agent2_grabbing = False
                p.removeConstraint(self.agent2_constraint_id)
                self.agent2_constraint_id = None
        

        reward = -dist * 0.1

        if self.attached:
            reward += 1.0  # bonus for attachment
            self.terminations[agent] =True # {a: True for a in self.agents}
        self.agent_idx = (self.agent_idx + 1) % len(self.agents)
        self.agent_selection = self._agent_selector.next()


    def render(self):
        pass

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]
    
    def create_wall(x_center, y_center, half_ext_x, half_ext_y, height=0.4):
        collision_shape = p.createCollisionShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[half_ext_x, half_ext_y, height / 2.0]
        )
        visual_shape = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[half_ext_x, half_ext_y, height / 2.0],
            rgbaColor=[0.8, 0.8, 0.8, 1]
        )
        body_id = p.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[x_center, y_center, height / 2.0]
        )
        return body_id

    def create_cube_block(pos_x, pos_y, size=0.2, mass=100.00):
        collision_shape = p.createCollisionShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[size/2, size/2, size/2]
        )
        visual_shape = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[size/2, size/2, size/2],
            rgbaColor=[0, 1, 0, 1]  # green
        )
        body_id = p.createMultiBody(
            baseMass=mass,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[pos_x, pos_y, size/2]
        )
        return body_id

    def create_point_agent(pos_x, pos_y, radius=0.05, mass=1.0, color=[1, 0, 0, 1]):
        collision_shape = p.createCollisionShape(
            shapeType=p.GEOM_SPHERE,
            radius=radius
        )
        visual_shape = p.createVisualShape(
            shapeType=p.GEOM_SPHERE,
            radius=radius,
            rgbaColor=color
        )
        body_id = p.createMultiBody(
            baseMass=mass,
            baseCollisionShapeIndex=collision_shape,
            baseVisualShapeIndex=visual_shape,
            basePosition=[pos_x, pos_y, radius]
        )
        return body_id

    def distance_between_bodies(bodyA, bodyB):
        """
        Returns Euclidean distance between the base positions of bodyA and bodyB.
        """
        posA, _ = p.getBasePositionAndOrientation(bodyA)
        posB, _ = p.getBasePositionAndOrientation(bodyB)
        dx = posA[0] - posB[0]
        dy = posA[1] - posB[1]
        dz = posA[2] - posB[2]
        return math.sqrt(dx*dx + dy*dy + dz*dz)
    # ─────────────────────────────────────────────────────────────────────────────
    # Helper: convert a world-space point to a body-local point
    # ─────────────────────────────────────────────────────────────────────────────
    def world_to_local(body_id, world_pt):
        base_pos, base_orn = p.getBasePositionAndOrientation(body_id)
        inv_pos, inv_orn   = p.invertTransform(base_pos, base_orn)
        local_pt, _        = p.multiplyTransforms(inv_pos, inv_orn,
                                                world_pt, [0, 0, 0, 1])
        return local_pt
    # ─────────────────────────────────────────────────────────────────────────────
    # Helper: project "agent → cube-centre" line onto the cube surface
    #         so we grab the *wall* not the centre.
    # ─────────────────────────────────────────────────────────────────────────────
    def closest_point_on_cube_surface(cube_id, query_pt):
        # get cube AABB in world coords
        aabb_min, aabb_max = p.getAABB(cube_id)

        # clamp query point to AABB to get the closest point *inside* the cube
        cp_inside = np.clip(query_pt, aabb_min, aabb_max)

        # now push that point to the nearest face, i.e. choose the axis on which
        # |difference| to the face is smallest and project to the face
        dists_to_min = np.abs(cp_inside - aabb_min)
        dists_to_max = np.abs(aabb_max - cp_inside)
        axis          = np.argmin(np.minimum(dists_to_min, dists_to_max))

        # move to the cube surface along that axis
        if cp_inside[axis] - aabb_min[axis] < aabb_max[axis] - cp_inside[axis]:
            cp_surface = cp_inside.copy()
            cp_surface[axis] = aabb_min[axis]      # snap to − face
        else:
            cp_surface = cp_inside.copy()
            cp_surface[axis] = aabb_max[axis]      # snap to + face
        return cp_surface.tolist()


    # ─────────────────────────────────────────────────────────────────────────────
    # The new constraint creator
    # ─────────────────────────────────────────────────────────────────────────────
    def create_fixed_constraint(bodyA, bodyB, grab_distance_threshold=0.25):
        """
        bodyA … the *agent*  (parent)
        bodyB … the *cube*   (child)

        Creates a rigid joint between the agent surface and the nearest cube wall.
        Returns the constraint id or None if the agent is too far away.
        """
        # 1️⃣ work in world space ---------------------------------------------------
        agent_pos, _ = p.getBasePositionAndOrientation(bodyA)
        cube_pos,  _ = p.getBasePositionAndOrientation(bodyB)

        # bail out if they are too far apart
        if np.linalg.norm(np.array(agent_pos) - np.array(cube_pos)) > grab_distance_threshold:
            return None

        # 2️⃣ pick the cube-wall point --------------------------------------------
        pivot_world = closest_point_on_cube_surface(bodyB, agent_pos)

        # 3️⃣ compute local frames for each body -----------------------------------
        parent_local = world_to_local(bodyA, pivot_world)
        child_local  = world_to_local(bodyB, pivot_world)

        # 4️⃣ make the joint --------------------------------------------------------
        cid = p.createConstraint(parentBodyUniqueId=bodyA, parentLinkIndex=-1,
                                childBodyUniqueId=bodyB,  childLinkIndex=-1,
                                jointType=p.JOINT_FIXED,  jointAxis=[0, 0, 0],
                                parentFramePosition=parent_local,
                                childFramePosition=child_local)

        # give it plenty of authority
        p.changeConstraint(cid, maxForce=1e6)
        return cid

